# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [2]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime

from functions import train_model

In [3]:
# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [4]:
portfolio_raw

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [ ]:
#portfolio_raw['duration']*24

In [5]:
portfolio_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [6]:
portfolio_raw['channels'].explode().unique()

array(['email', 'mobile', 'social', 'web'], dtype=object)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [7]:
profile_raw.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [8]:
profile_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [9]:
profile_raw.describe(include='all')

,gender,age,id,became_member_on,income
count,14825,17000.000000,17000,1.700000e+04,14825.000000
unique,3,NaN,17000,NaN,NaN
top,M,NaN,e4052622e5ba45a8b96b59aba68cf068,NaN,NaN
freq,8484,NaN,1,NaN,NaN
mean,NaN,62.531412,NaN,2.016703e+07,65404.991568
std,NaN,26.738580,NaN,1.167750e+04,21598.299410
min,NaN,18.000000,NaN,2.013073e+07,30000.000000
25%,NaN,45.000000,NaN,2.016053e+07,49000.000000
50%,NaN,58.000000,NaN,2.017080e+07,64000.000000
75%,NaN,73.000000,NaN,2.017123e+07,80000.000000


In [10]:
profile_raw['gender'].value_counts(dropna=False)

gender
M       8484
F       6129
None    2175
O        212
Name: count, dtype: int64

In [11]:
profile_raw[profile_raw['gender'].isna()]['age'].value_counts(dropna=False)

age
118    2175
Name: count, dtype: int64

In [12]:
profile_raw[profile_raw['gender'].isna()]['income'].value_counts(dropna=False)

income
NaN    2175
Name: count, dtype: int64

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [13]:
transcript_raw.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [14]:
transcript_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [15]:
transcript_raw['value']

0         {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
306529                      {'amount': 1.5899999999999999}
306530                                    {'amount': 9.53}
306531                                    {'amount': 3.61}
306532                      {'amount': 3.5300000000000002}
306533                                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object

In [16]:
transcript_raw['event'].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [17]:
set([list(x.keys())[0] for x in transcript_raw['value']])

{'amount', 'offer id', 'offer_id'}

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [18]:
#storage profiles with noneType gender values in separeted files
profile_none = profile_raw[profile_raw['gender'].isna()].copy()
profile = profile_raw[~profile_raw['gender'].isna()].copy()
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')


# change the id GUIDs (Globally Unique Identifiers) to short tag.
port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

portfolio = portfolio_raw.copy()
portfolio['id_short'] = portfolio_raw['id'].map(port_id)

# Column renaming
transcript = transcript_raw.copy()
transcript['event'] = transcript_raw['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)


# creating a dataframe handling the data in 'value' column
value_df = pd.json_normalize(transcript['value']).fillna(0)
value_df['offer_id'] = value_df['offer_id'].map(port_id).fillna('tran')
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

In [19]:
# cheking profile 
profile.describe(include='all').round(0) 

,gender,age,id,became_member_on,income
count,14825,14825.0,14825,14825,14825.0
unique,3,NaN,14825,NaN,NaN
top,M,NaN,e4052622e5ba45a8b96b59aba68cf068,NaN,NaN
freq,8484,NaN,1,NaN,NaN
mean,NaN,54.0,NaN,2017-02-18 12:30:15.419898880,65405.0
min,NaN,18.0,NaN,2013-07-29 00:00:00,30000.0
25%,NaN,42.0,NaN,2016-05-20 00:00:00,49000.0
50%,NaN,55.0,NaN,2017-08-02 00:00:00,64000.0
75%,NaN,66.0,NaN,2017-12-30 00:00:00,80000.0
max,NaN,101.0,NaN,2018-07-26 00:00:00,120000.0


In [20]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   gender            14825 non-null  object        
 1   age               14825 non-null  int64         
 2   id                14825 non-null  object        
 3   became_member_on  14825 non-null  datetime64[ns]
 4   income            14825 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 694.9+ KB


In [21]:
# Checking the data transcript_b created
transcript_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   person    306534 non-null  object 
 1   event     306534 non-null  object 
 2   time      306534 non-null  int64  
 3   offer_id  306534 non-null  object 
 4   amount    306534 non-null  float64
 5   reward    306534 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 14.0+ MB


# Grouping/Merging Datasets and Features Engineering

**task:**
### Creating the `person_activity` Table: Customer Interaction Insights  

The goal is to construct the `person_activity` table to analyze customer behavior by tracking their event timelines and generating relevant time-based features.

This structured dataset enables detailed analysis of how customers interact with offers, their responsiveness, and purchasing behaviors over time.  

---

#### **Objective**  
Track and organize customer interactions (`person`, `event`, `offer_id`) into a structured `timeline`, generating features that offer insight into customer behavior.  

---

#### **Steps**  

1. **Create the `tag` column:**  
   - Enumerate unique combinations of `['person', 'event', 'offer_id']` to identify distinct events.  

2. **Group and unstack:**  
   - Group data by `person`, `offer_id`, and `tag`, and unstack the event timeline to create a structured timeline for customer interactions.  

3. **Merge Transaction Data:**  
   - Include transactional data by linking `time` and `amount` fields with the timeline structure.  

4. **Generate Count-Based Features:**  
   - Count key events (`offer_received`, `offer_viewed`, `offer_completed`) for each customer and offer.  
   - Add features to measure transaction-related behaviors such as the number of transactions.

5. **Calculate Time-Based Features:**  
   - Compute the time differences between events (`offer_received`, `offer_viewed`, and `offer_completed`) to assess responsiveness and engagement.  

6. **Behavioral Metrics:**  
   - Develop derived metrics to measure curiosity, eagerness, and overall responsiveness using the time differences.  

7. **Influence Indicators:**  
   - Identify whether customers were influenced by the offers based on the relationship between event timestamps.  

8. **Additional Features and Cleaning:**  
   - Compute average engagement time and handle missing values for transaction-related features like `amount` and `number of transactions`.  

---

#### **Outcome**  

The `person_activity` table will contain:  

1. **Timeline Events** for each `person` and `offer_id`:  
   - Key columns include `offer_received`, `offer_viewed`, and `offer_completed`.  

### Derived Features  

1. **Count-Based Features:**  
   - `n_rec`: Number of received offers.  
   - `n_vie`: Number of viewed offers.  
   - `n_com`: Number of completed offers.  
   - `n_tran`: Number of transactions.  
   - `amt_mean`: Average transaction amount.  

2. **Time-Based Metrics:**  
   - `to_vr`: Time between viewing and receiving an offer.  
   - `to_cv`: Time between completing and viewing an offer.  
   - `to_cr`: Time between completing and receiving an offer.  

3. **Behavioral Metrics:**  
   - `curiosity_vr`: Metric reflecting curiosity (based on time to view).  
   - `eagerness_cv`: Metric reflecting eagerness (based on time to complete).  
   - `overall_cr`: Overall responsiveness (based on time to complete after receiving).  

4. **Influence Indicators:**  
   - `ofr_influence`: Indicates if completion happened after viewing.  
   - `extrm_ofr_influence`: Indicates if completion occurred immediately after viewing.  



In [22]:
# creating 'tag' column
transcript_b['tag'] = (
    transcript_b.groupby(['person', 'event', 'offer_id'])
    .cumcount()
)


# Group and unstack the transcript_b dataframe
person_activity = (
    transcript_b[transcript_b['event'] != 'transaction']
    .groupby(['person', 'offer_id', 'tag', 'event'])['time']
    .first()
    .unstack(fill_value=5000)
    .reset_index()
)

# Filtering the transcript_b dataframe to only include transactions
transactions = transcript_b.loc[transcript_b['event'] == 'transaction', ['person', 'time', 'amount']]


# Merge the person_activity dataframe with the transactions dataframe
person_activity = pd.merge(
    how='left',
    left=person_activity,
    right=transactions,
    left_on=['person', 'offer_completed'],
    right_on=['person', 'time'],
).drop(columns=['time'])


# Creating count features using the groupby() function and the agg() function. Merge the result with the person_activity dataframe
aggregations1 = person_activity.groupby(['person', 'offer_id']).agg(
    n_rec=('offer_received', 'count'),
    n_vie=('offer_viewed', 'count'),
    n_com=('offer_completed', 'count'),

).reset_index()

person_activity = person_activity.merge(aggregations1, on=['person', 'offer_id'], how='left')


aggregations2 = transcript_b[transcript_b['event'] == 'transaction'].groupby(['person', 'offer_id']).agg(
    n_tran = ('event', 'count'),
    amt_mean = ('amount', 'mean')

)

person_activity = person_activity.merge(aggregations2, on=['person'], how='left')


# Creating other features directly from the person_activity dataframe

person_activity['to_vr'] = person_activity['offer_viewed'] - person_activity['offer_received']
person_activity['to_cv'] = person_activity['offer_completed'] - person_activity['offer_viewed'] 
person_activity['to_cr'] = person_activity['offer_completed'] - person_activity['offer_received']

person_activity['curiosity_vr'] = (1 / (person_activity['to_vr'] + 1)) * 100
person_activity['eagerness_cv'] = (1 / (person_activity['to_cv'] + 1)) * 100
person_activity['overall_cr'] = (1 / (person_activity['to_cr'] + 1)) * 100

person_activity['ofr_influence'] = ((person_activity['offer_completed'] != 1000) & (person_activity['offer_completed'] >= person_activity['offer_viewed'])) 
person_activity['extrm_ofr_influence'] = ((person_activity['offer_viewed'] != 1000) & (person_activity['offer_completed'] == person_activity['offer_viewed']))

person_activity['avg_time_to_engage'] = (
    (person_activity['to_vr'] + person_activity['to_cv']) / 2
)


person_activity.loc[:,'amount'] = person_activity.loc[:,'amount'].fillna(0)
person_activity.loc[:,'n_tran'] = person_activity.loc[:,'n_tran'].fillna(0)
person_activity.loc[:,'amt_mean'] = person_activity.loc[:,'amt_mean'].fillna(0)

person_activity = person_activity.round(2)

person_activity

,person,offer_id,tag,offer_completed,offer_received,offer_viewed,amount,n_rec,n_vie,n_com,...,amt_mean,to_vr,to_cv,to_cr,curiosity_vr,eagerness_cv,overall_cr,ofr_influence,extrm_ofr_influence,avg_time_to_engage
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,5000,336,372,0.00,1,1,1,...,15.95,36,4628,4664,2.70,0.02,0.02,True,False,2332.0
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,528,504,540,14.11,1,1,1,...,15.95,36,-12,24,2.70,-9.09,4.00,False,False,12.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,5000,168,192,0.00,1,1,1,...,15.95,24,4808,4832,4.00,0.02,0.02,True,False,2416.0
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,414,408,456,8.57,1,1,1,...,15.95,48,-42,6,2.04,-2.44,14.29,False,False,3.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,576,576,5000,10.27,1,1,1,...,15.95,4424,-4424,0,0.02,-0.02,100.00,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,198,168,174,22.88,1,1,1,...,15.07,6,24,30,14.29,4.00,3.23,True,False,15.0
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,60,0,6,16.06,1,1,1,...,15.07,6,54,60,14.29,1.82,1.64,True,False,30.0
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,384,336,354,15.57,3,3,3,...,15.07,18,30,48,5.26,3.23,2.04,True,False,24.0
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,414,408,414,17.55,3,3,3,...,15.07,6,0,6,14.29,100.00,14.29,True,True,3.0


In [23]:
person_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   person               76277 non-null  object 
 1   offer_id             76277 non-null  object 
 2   tag                  76277 non-null  int64  
 3   offer_completed      76277 non-null  int64  
 4   offer_received       76277 non-null  int64  
 5   offer_viewed         76277 non-null  int64  
 6   amount               76277 non-null  float64
 7   n_rec                76277 non-null  int64  
 8   n_vie                76277 non-null  int64  
 9   n_com                76277 non-null  int64  
 10  n_tran               76277 non-null  float64
 11  amt_mean             76277 non-null  float64
 12  to_vr                76277 non-null  int64  
 13  to_cv                76277 non-null  int64  
 14  to_cr                76277 non-null  int64  
 15  curiosity_vr         76277 non-null 

In [24]:
person_activity[person_activity['person'] == 'ffff82501cea40309d5fdd7edcca4a07']

,person,offer_id,tag,offer_completed,offer_received,offer_viewed,amount,n_rec,n_vie,n_com,...,amt_mean,to_vr,to_cv,to_cr,curiosity_vr,eagerness_cv,overall_cr,ofr_influence,extrm_ofr_influence,avg_time_to_engage
76271,ffff82501cea40309d5fdd7edcca4a07,ofr_D,0,504,504,534,7.79,1,1,1,...,15.07,30,-30,0,3.23,-3.45,100.00,False,False,0.0
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,198,168,174,22.88,1,1,1,...,15.07,6,24,30,14.29,4.00,3.23,True,False,15.0
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,60,0,6,16.06,1,1,1,...,15.07,6,54,60,14.29,1.82,1.64,True,False,30.0
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,384,336,354,15.57,3,3,3,...,15.07,18,30,48,5.26,3.23,2.04,True,False,24.0
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,414,408,414,17.55,3,3,3,...,15.07,6,0,6,14.29,100.00,14.29,True,True,3.0
76276,ffff82501cea40309d5fdd7edcca4a07,ofr_J,2,576,576,582,14.23,3,3,3,...,15.07,6,-6,0,14.29,-20.00,100.00,False,False,0.0


In [25]:
grouped1 = transcript_b[transcript_b['event'] == 'offer_completed'].groupby(['person', 'offer_id'])
grouped2 = transcript_b[transcript_b['event'] == 'transaction'].groupby(['person', 'offer_id'])

aggregations1 = grouped1.agg(
    offer_fidelity=('event', 'count'),
    first_comp =('time', 'min'), 
    last_comp =('time', 'max'), 
).reset_index()

aggregations2 = grouped2.agg(
    fidelity_tran=('event', 'count'),
    first_time_tr=('time', 'min'),
    last_time_tr=('time', 'max')

).reset_index()

fidelity = pd.merge(
    how='left',
    left=aggregations1,
    right=aggregations2,
    on=['person']
)

fidelity['lst_time_to_end_tr'] = transcript_b['time'].max() - fidelity['last_time_tr']
fidelity['lst_time_to_end_com'] = transcript_b['time'].max() - fidelity['last_comp']

fidelity

,person,offer_id_x,offer_fidelity,first_comp,last_comp,offer_id_y,fidelity_tran,first_time_tr,last_time_tr,lst_time_to_end_tr,lst_time_to_end_com
0,0009655768c64bdeb2e877511632db8f,ofr_G,1,528,528,tran,8,228,696,18,186
1,0009655768c64bdeb2e877511632db8f,ofr_I,1,414,414,tran,8,228,696,18,300
2,0009655768c64bdeb2e877511632db8f,ofr_J,1,576,576,tran,8,228,696,18,138
3,0011e0d4e6b944f998e987f904e8c1e5,ofr_D,1,576,576,tran,5,132,654,60,138
4,0011e0d4e6b944f998e987f904e8c1e5,ofr_E,1,576,576,tran,5,132,654,60,138
...,...,...,...,...,...,...,...,...,...,...,...
28991,fffad4f4828548d1b5583907f2e9906b,ofr_I,2,36,516,tran,12,36,678,36,198
28992,ffff82501cea40309d5fdd7edcca4a07,ofr_D,1,504,504,tran,15,60,648,66,210
28993,ffff82501cea40309d5fdd7edcca4a07,ofr_E,1,198,198,tran,15,60,648,66,516
28994,ffff82501cea40309d5fdd7edcca4a07,ofr_G,1,60,60,tran,15,60,648,66,654


In [26]:
fidelity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28996 entries, 0 to 28995
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   person               28996 non-null  object
 1   offer_id_x           28996 non-null  object
 2   offer_fidelity       28996 non-null  int64 
 3   first_comp           28996 non-null  int64 
 4   last_comp            28996 non-null  int64 
 5   offer_id_y           28996 non-null  object
 6   fidelity_tran        28996 non-null  int64 
 7   first_time_tr        28996 non-null  int64 
 8   last_time_tr         28996 non-null  int64 
 9   lst_time_to_end_tr   28996 non-null  int64 
 10  lst_time_to_end_com  28996 non-null  int64 
dtypes: int64(8), object(3)
memory usage: 2.4+ MB


In [234]:
def categorize_age(age):
    if age < 25:
        return 'Young Adult'
    elif 25 <= age < 40:
        return 'Adult'
    elif 40 <= age < 60:
        return 'Middle-Aged'
    else:
        return 'Senior'

# Aplicar a função ao DataFrame
tb_transactions['age_segment'] = tb_transactions['age'].apply(categorize_age)

In [38]:
#df = df.dropna()

X = df.drop(columns = ['person', 'event', 'became_member_on', 'gender', 'income'])
y = df['gender']

In [ ]:
y.value_counts()

In [ ]:
model1 = RandomForestClassifier(
    random_state=42,
    max_depth=15,
    n_estimators=25,
    min_samples_leaf=2,
    min_samples_split=2,
    #max_features=0.5,
    n_jobs=-1    
    )

model2 = DecisionTreeClassifier(
    random_state=42,
    criterion='entropy',            # Função de impureza para divisão ('gini' ou 'entropy')
    max_depth=15,                # Profundidade máxima da árvore (controle de overfitting)
    min_samples_split=2,         # Número mínimo de amostras para dividir um nó
    min_samples_leaf=2,          # Número mínimo de amostras em uma folha
    #max_features=5,           # Número máximo de features consideradas em cada divisão
    #splitter='random'            # Estratégia para escolher a divisão ('best' ou 'random')
    )



In [ ]:
model_type = model2
train_model(X, y, model_type)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Instanciar o modelo
rf = RandomForestClassifier(random_state=42)

# Definir a grade de parâmetros
param_grid = {
    'n_estimators': [50, 100, 200],         # Número de árvores na floresta
    'max_depth': [None, 10, 20, 30],       # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],       # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],         # Número mínimo de amostras em uma folha
    'max_features': ['sqrt', 'log2'],      # Número de recursos considerados para divisão
    'bootstrap': [True, False]             # Amostragem com ou sem reposição
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                  # Validação cruzada (5-fold)
    scoring='accuracy',    # Métrica de avaliação
    verbose=2,             # Nível de detalhe das mensagens
    n_jobs=-1              # Paralelização (usar todos os núcleos disponíveis)
)

# Ajustar o modelo aos dados de treino
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros e a melhor pontuação
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor pontuação (treino):", grid_search.best_score_)

# Avaliar o modelo com os melhores parâmetros nos dados de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Acurácia nos dados de teste:", test_accuracy)


y_pred_train = best_model.predict(X_train)
y_pred = best_model.predict(X_test)

# 1. Confusion Matrix
cm1 = confusion_matrix(y_train, y_pred_train)
cm2 = confusion_matrix(y_test, y_pred)


# 2. Relatório com Precision, Recall e F1-Score
report1 = classification_report(y_test, y_pred, target_names=best_model.classes_, zero_division=1)
print("Test Classification Report:")
print(report1)

report2 = classification_report(y_train, y_pred_train, target_names=best_model.classes_, zero_division=1)
print("Train Classification Report:")
print(report2)